Importing required libraries and preprocessing the text

In [2]:
import re 
import spacy 
import nltk
from nltk.tokenize import sent_tokenize
from nltk.corpus import stopword


nltk.download('punkt')
nltk.download('stopword')

nlp = spacy.load("en_core_web_sm")
stop_words = set(stopwords.words('english'))

def preprocess_text(text):
    # removing unnecessary characters, stopwords and tokes in senctences
    text = re.sub(r"[^a-zA-Z0-9\s]" , "", text)  #removeing punctuation
    sentences = sent_tokenize(text) #tokenizing sentences
    clean_sentences = []

    
    for sentence in sentences:
        tokens = [word.lower()
                  for word in sent.split()
                  if word.lower() not in stop_words]
        clean_sentences.append("".join(tokens))
        
        
    return clean_sentences

ModuleNotFoundError: No module named 'nltk.tokenze'

Task Indentification